In [67]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [68]:
llm = ChatGroq(
    temperature=0,
    model = "llama-3.3-70b-versatile"
)

response = llm.invoke("the first person to land on the moon was.....")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [69]:
# %pip install langchain langchain-community beautifulsoup4


In [70]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.adidas-group.com/")
page_data = loader.load().pop().page_content
print(page_data)







                            adidas Careers – Through sport, we have the power to change lives.
                    















































English
German
French




The Company


Teams




                                    Makers
                                


Design


Digital


Engineering


Marketing & Communications


Product Development


Production


Research & Development





                                    Growers
                                


Accounting & Finance


Administrative


Audit


Customer Service


Facilities & Services


Human Resources


Tech


Legal / Social Environmental Affairs


Sales


Sourcing


Strategy


Supply Chain Management





                                    Retailers
                                


Retail Back Office


Retail Store


Our Stores


Retail Career Opportunities


 





                                    Pupils
                                


Pupil Internship


Apprenticeship Program

In [71]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format.
    
    The JSON should be a SINGLE DICTIONARY with a key "jobs" containing a list of the postings.
    Each posting in the list should have the following keys: `role`, `experience`, `skills` and `description`.
    
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [72]:
# convert str to JSON
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content) # ignore 
json_res

{'jobs': [{'role': 'Manager Brand Communications BU Originals, Basketball & Partnerships',
   'experience': None,
   'skills': None,
   'description': 'Tokyo, Japan, Brand Management & Communications, December 12 2025 - 538089'},
  {'role': 'DC / Distribution Lead (Training) ~ Day (S) Shift, Mon - Fri 7:00am - 3:30pm',
   'experience': None,
   'skills': None,
   'description': 'Spartanburg, United States of America, Supply Chain & Sourcing, December 12 2025 - 538351'},
  {'role': 'Sr. Specialist, Retail Marketing - Golf Brand',
   'experience': None,
   'skills': None,
   'description': 'Seoul, Korea, Republic of, Brand Management & Communications, December 12 2025 - 538226'},
  {'role': 'Stockroom Associate (m/f/d) 39h vollzeit, befristet -BCS Herzogenaurach',
   'experience': None,
   'skills': None,
   'description': 'Herzogenaurach, Germany, Retail, December 12 2025 - 538396'},
  {'role': 'Assistant Manager Sport Performance, Wholesale',
   'experience': None,
   'skills': None,
 

In [73]:
type(json_res)

dict

In [74]:
import pandas as pd
df = pd.read_csv("my_portfolio.csv")
df


,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [75]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorestore')
collection = client.get_or_create_collection(name = 'portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                        metadatas ={"links":row["Links"]},
                        ids = [str(uuid.uuid4())])
        

In [76]:
# Wrap res.content in str() to satisfy the type checker
json_res = json_parser.parse(str(res.content))

In [77]:
# Manually define the job dictionary with your specific skills
job = {
    'skills': ["Python", "Streamlit", "Machine Learning", "LangChain"]
}

# Now your query line will work
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/kotlin-android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}]]

In [78]:
for job in json_res['jobs']:
    
    # 1. Get skills, defaulting to empty list
    skills = job.get('skills', [])
    
    # 2. logical check: If skills list is empty, use defaults
    if not skills:
        skills = ["Python", "Generative AI", "React", "Data Science"]
    
    # 3. Query the portfolio (using the 'skills' variable)
    # query_texts MUST NOT be empty
    links = collection.query(query_texts=skills, n_results=2).get('metadatas', [])
    
    print(f"Links for job {job['role']}: {links}")

Links for job Manager Brand Communications BU Originals, Basketball & Partnerships: [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}]]
Links for job DC / Distribution Lead (Training) ~ Day (S) Shift, Mon - Fri 7:00am - 3:30pm: [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': '

In [79]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Raunak, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    
    Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Sales Planning Solutions with AtliQ

Dear Hiring Manager,

I came across the job posting for Sales Planner (7 Positions Available) at your esteemed organization in Portland, United States of America. As a business development executive at AtliQ, I am excited to introduce our company's capabilities in providing tailored solutions to facilitate seamless integration of business processes through automated tools.

At AtliQ, we have empowered numerous enterprises with our expertise in AI and software consulting, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can help you streamline your sales planning process, leveraging data-driven insights to drive business growth.

Our portfolio showcases our expertise in machine learning and Python, with notable projects including:
https://example.com/ml-python-portfolio and https://example.com/python-portfolio. These examples demonstrate our ability to develop and impl